In [0]:
import pandas as pd
import numpy as np


In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")

In [0]:
t= train["Y18"].isnull()
index_null = t[t==True].index

In [0]:
train.loc[index_null,"Y18"] = train.loc[index_null,"Y16"]

In [0]:
train = pd.concat([train,train],axis=0)

In [0]:
test = test.loc[:,test.min() != test.max()]
train = train.loc[:,train.min() != train.max()]

In [0]:
x=train.loc[:,"X00":"X39"]
y=train.loc[:,"Y18"]

In [0]:
def standardization(df):
    mean = np.mean(df)
    std = np.std(df)
    norm = (df - mean) / (std - 1e-07)
    return norm, mean, std

In [0]:
X_train_norm, mean, std = standardization(x)
X_test = (test -mean) / (std - 1e-07)

In [0]:
# ID로 시간 변수 생성
minute = pd.Series((X_train_norm.index%144).astype(int))
hour= pd.Series((X_train_norm.index%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)

In [0]:
X_train_norm['minute_sin'] = minute_sin
X_train_norm['minute_cos'] = minute_cos

X_train_norm['hour_sin'] = hour_sin
X_train_norm['hour_cos'] = hour_cos
X_train_norm

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,minute_sin,minute_cos,hour_sin,hour_cos
0,-2.236613,0.053249,-0.289028,-0.911441,-0.286772,0.284592,0.174800,-2.160980,0.315249,0.396477,-0.302862,-1.012516,0.637025,-0.911726,-0.749131,-0.911954,-0.932947,0.894310,-0.279106,0.215838,0.277710,0.105335,-1.044134,-0.701087,0.332281,-2.314698,0.232479,0.159935,-2.110001,-1.766809,0.298012,-1.015988,0.482067,-0.274975,0.577780,0.240728,-0.234601,0.000000,1.000000,0.000000,1.000000
1,-2.321591,0.083015,0.177501,0.477370,-0.286772,0.284592,0.174800,-2.183828,0.343658,0.424679,-0.302862,-1.012516,0.583154,-0.400474,-0.092525,-1.395861,-0.846359,0.935541,-0.279106,0.215838,0.306973,-0.902731,-1.061634,-1.104062,0.332281,-2.362038,0.261979,0.215689,-2.091884,-1.842695,0.298012,-1.015988,0.070297,-0.274975,0.582902,0.289731,-0.234601,0.021815,0.999762,0.000000,1.000000
2,-2.300347,0.112780,-0.382334,0.904697,-0.286772,0.255134,0.204183,-2.183828,0.372068,0.481084,-0.302862,-1.012516,0.803125,0.715170,-0.310371,-1.395861,-0.673184,0.999678,-0.279106,0.186142,0.277710,-0.902731,-1.061634,-1.104062,0.303262,-2.362038,0.232479,0.271442,-2.146235,-1.956524,0.268945,-1.015988,0.269152,-0.274975,0.582902,0.311510,-0.234601,0.043619,0.999048,0.000000,1.000000
3,-2.300347,0.083015,-0.009111,-0.804610,-0.286772,0.255134,0.174800,-2.206676,0.343658,0.452881,-0.302862,-1.012516,0.874952,-1.633554,-0.597764,1.140988,-1.106121,1.004260,-0.279106,0.186142,0.248447,-0.902731,-1.061634,-0.942872,0.332281,-2.362038,0.202979,0.350426,-2.200586,-2.013438,0.298012,-1.015988,0.056237,-0.274975,0.593145,0.344178,-0.234601,0.065403,0.997859,0.000000,1.000000
4,-2.342836,0.083015,-0.662251,0.263707,-0.286772,0.255134,0.204183,-2.206676,0.372068,0.452881,-0.302862,-1.012516,0.928822,-1.633554,-0.507762,-1.395861,-1.192709,1.036328,-0.279106,0.186142,0.248447,-0.902731,-1.061634,-1.104062,0.332281,-2.385708,0.202979,0.401533,-2.254937,-2.013438,0.298012,-1.015988,-0.336452,-0.274975,0.618753,0.371402,-0.234601,0.087156,0.996195,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,-0.069662,-0.303940,-0.568945,-0.697778,-0.286772,-0.422397,-0.383472,0.009578,-0.366580,-0.280379,0.449229,1.183872,1.216131,1.738687,1.028410,0.980548,-0.413422,0.834754,2.295301,-0.407792,-0.395341,-0.902731,-1.061634,-0.136920,-0.451211,0.407351,-0.416518,1.126324,-0.388886,-0.173206,-0.457735,0.930342,0.102435,-0.274975,0.838982,0.022938,-0.126302,0.108867,-0.994056,0.130526,-0.991445
4748,-0.069662,-0.303940,-0.942168,-0.804610,-0.286772,-0.451855,-0.383472,-0.013270,-0.366580,-0.280379,0.449229,1.183872,1.229598,0.557238,1.052956,0.734713,-0.326835,0.853079,2.295301,-0.437489,-0.395341,-0.902731,-1.061634,-0.056325,-0.451211,0.360011,-0.416518,1.191369,-0.388886,-0.154235,-0.457735,0.930342,-0.469021,-0.274975,0.849226,0.050162,-0.126302,0.087156,-0.996195,0.130526,-0.991445
4749,-0.112151,-0.274175,-0.568945,-0.911441,-0.286772,-0.422397,-0.412855,-0.058966,-0.394989,-0.252177,0.449229,1.183872,1.243066,-0.339731,0.836133,1.184117,-0.673184,0.885147,2.295301,-0.407792,-0.366078,-0.902731,-1.061634,-0.942872,-0.451211,0.360011,-0.387018,1.140262,-0.370769,-0.154235,-0.457735,0.930342,0.459972,-0.274975,0.874834,0.039273,-0.126302,0.065403,-0.997859,0.130526,-0.991445
4750,-0.175885,-0.274175,-0.568945,-0.697778,-0.286772,-0.422397,-0.354090,-0.081814,-0.338170,-0.252177,0.449229,1.183872,1.243066,1.644535,0.825906,-1.395861,-0.673184,0.917216,2.295301,-0.407792,-0.366078,-0.902731,-1.061634,-1.023467,-0.451211,0.289001,-0.387018,1.135616,-0.388886,-0.135263,-0.428667,0.930342,0.919950,-0.274975,0.885077,0.088275,-0.126302,0.043619,-0.999048,0.130526,-0.991445


In [0]:
test

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39
0,4752,18.7,987.4,1.5,1.3,0.0,1006.8,987.7,21.1,1007.5,1007.2,0.0,0.0,96.1,333.6,199.6,36.0,1.1,94.7,0.0,1000.3,1007.6,0.0,0.0,0.1,1006.0,23.2,999.5,91.4,17.5,19.2,1007.6,0.00,194.7,0.0,84.0,59.9,0.0
1,4753,18.8,987.4,1.1,1.2,0.0,1006.7,987.7,21.2,1007.5,1007.2,0.0,0.0,96.2,335.5,194.0,127.7,0.6,95.3,0.0,1000.2,1007.5,0.0,0.0,0.3,1005.9,23.1,999.4,91.6,17.4,19.1,1007.5,0.00,173.1,0.0,84.0,60.5,0.0
2,4754,19.0,987.4,1.3,1.2,0.0,1006.6,987.6,21.2,1007.4,1007.2,0.0,0.0,96.4,0.0,172.9,59.0,0.0,93.9,0.0,1000.1,1007.4,0.0,0.0,0.7,1005.9,23.1,999.3,92.0,17.2,19.0,1007.5,0.00,208.6,0.0,85.0,60.8,0.0
3,4755,18.7,987.4,2.3,0.8,0.0,1006.6,987.6,21.1,1007.4,1007.2,0.0,0.0,96.5,252.9,153.5,54.9,0.4,94.4,0.0,1000.1,1007.4,0.0,0.0,0.6,1005.8,22.8,999.3,93.0,17.2,18.8,1007.4,0.00,185.0,0.0,85.8,61.2,0.0
4,4756,18.4,987.4,1.1,0.7,0.0,1006.7,987.7,20.9,1007.5,1007.2,0.0,0.0,96.6,238.2,173.8,48.8,0.8,95.7,0.0,1000.2,1007.3,0.0,0.0,0.7,1005.9,22.8,999.2,92.3,17.4,18.9,1007.5,0.00,162.8,0.0,87.3,61.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,16267,25.7,993.2,3.9,3.8,0.0,1011.7,993.8,26.3,1013.3,1012.6,0.0,17.1,88.1,65.6,101.6,176.7,1.8,77.3,0.0,1005.2,1012.9,1.6,169.8,4.5,1011.5,27.7,1004.9,76.6,23.6,24.4,1013.0,19.56,169.6,0.0,79.7,67.5,0.5
11516,16268,25.6,993.2,3.0,3.0,0.0,1011.7,993.8,26.3,1013.3,1012.6,0.0,17.1,75.5,51.9,84.8,187.8,2.9,78.1,0.0,1005.2,1012.9,1.9,138.9,4.7,1011.4,27.6,1004.9,76.7,25.3,24.4,1012.9,19.56,175.9,0.0,79.6,67.9,0.5
11517,16269,25.6,993.2,3.8,2.5,0.0,1011.9,993.7,26.2,1013.2,1012.7,0.0,17.1,77.5,29.0,104.5,184.8,2.9,78.9,0.0,1005.4,1013.0,2.3,146.7,4.8,1011.4,27.5,1005.0,77.1,24.9,24.3,1012.9,19.56,204.7,0.0,79.2,68.3,0.5
11518,16270,25.5,993.2,2.4,2.8,0.0,1011.8,993.7,26.1,1013.2,1012.6,0.0,17.1,74.1,30.0,103.0,187.8,3.4,79.7,0.0,1005.3,1013.0,1.7,146.5,3.5,1011.4,27.3,1005.0,77.3,25.3,24.2,1012.9,19.56,174.2,0.0,80.1,68.8,0.5


In [0]:

# ID로 시간 변수 생성
minute = pd.Series((test.id%144).astype(int))
hour= pd.Series((test.id%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)

In [0]:
X_test['minute_sin'] = minute_sin
X_test['minute_cos'] = minute_cos

X_test['hour_sin'] = hour_sin
X_test['hour_cos'] = hour_cos
X_test = X_test.drop(columns='id')

In [0]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

x_train, x_test, y_train, y_test = train_test_split(X_train_norm,y, train_size=0.7, shuffle=False)

In [0]:
def mse1(y_pred, dataset):
    y_true = dataset.get_label()

    diff = abs(y_true - y_pred)
    less_then_one = np.array([0 if x < 1 else 1 for x in diff])

    y_pred = less_then_one * y_pred
    y_true = less_then_one *y_true
    
    score= mean_squared_error(y_true, y_pred)
    
    return 'score', score, False

In [0]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb

from sklearn.metrics import r2_score, mean_squared_error

params={ 'max_depth': 3, 'n_estimators':5000,  'learning_rate': 0.007, \
        'boost_from_average': False, 'subsample': 1, 'loss_function':'RMSE', 'scale_pos_weight': 10, 'eta':1,'slient':1, 'nthread':10,
        'coal_sample_bytree':'0.75'}

clf= xgb.XGBRegressor(**params,feval=mse1)
#clf = cb.CatBoostRegressor(**params)
model=clf.fit(x_train,y_train, verbose=100)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:27:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [0]:
pred = model.predict(X_test)

In [0]:
print(pred.max())
print(train.Y16.max())

41.58121
41.5


In [0]:
sub['Y18'] = pred
sub.to_csv('result13.csv', index=False)

In [0]:
train

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.00,82.4,71.3,101.1,56.1,0.3,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.00,256.4,0.0,77.2,62.6,0.0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,8.0
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.00,81.2,121.8,165.3,0.0,0.4,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.00,215.4,0.0,77.3,63.5,0.0,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,7.5
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.00,86.1,232.0,144.0,0.0,0.6,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.00,235.2,0.0,77.3,63.9,0.0,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,7.5
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.00,87.7,0.0,115.9,294.1,0.1,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.00,214.0,0.0,77.5,64.5,0.0,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,7.5
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.00,88.9,0.0,124.7,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.00,174.9,0.0,78.0,65.0,0.0,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,4747,19.9,987.6,0.9,0.8,0.0,1006.9,987.7,21.7,1007.5,1007.4,2.5,23.88,95.3,333.1,274.9,275.5,0.9,89.7,10.5,1000.4,1007.5,0.0,0.0,1.2,1006.0,23.8,999.4,89.9,17.7,19.1,1007.5,22.16,218.6,0.0,82.3,58.6,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4748,4748,19.9,987.6,0.5,0.7,0.0,1006.8,987.7,21.6,1007.5,1007.4,2.5,23.88,95.6,216.4,277.3,247.0,1.0,90.1,10.5,1000.3,1007.5,0.0,0.0,1.3,1006.0,23.6,999.4,91.3,17.7,19.2,1007.5,22.16,161.7,0.0,82.5,59.1,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4749,4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,1007.5,2.5,23.88,95.9,127.8,256.1,299.1,0.6,90.8,10.5,1000.4,1007.6,0.0,0.0,0.2,1006.0,23.6,999.5,90.2,17.8,19.2,1007.5,22.16,254.2,0.0,83.0,58.9,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4750,4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,1007.5,2.5,23.88,95.9,323.8,255.1,0.0,0.6,91.5,10.5,1000.4,1007.6,0.0,0.0,0.1,1006.0,23.3,999.5,90.1,17.7,19.3,1007.6,22.16,300.0,0.0,83.2,59.8,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
